In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from collections import Counter

In [2]:
def scrape_one_new(html, url):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)

    # title, date, genre, keyword
    title = json_data.get('headline', soup.find('span', class_='contentTitle').text)
    date = json_data.get('datePublished', re.search(r'datetime:.*?(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})', str(html)).group(1))
    date_m = json_data.get('dateModified', '')
    genre = json_data.get('genre', [])
    if genre == []:
        genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB']]
    keywords = json_data.get('keywords', [])
    
    # article: news_textbody > news_textmore > news_add (paragraph titles are h3)
    article = soup.find('div', id="news_textbody").text
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title.strip(),
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url,
        'datePublished':date,
        'dateModified':date_m
    }

# for old web normal
def make_datetime_normal_old(hmd, time):
    year, month, day = hmd[:4], hmd[4:6], hmd[6:]
    hour, minute = time.split('時')
    minute = minute.strip('分')
    if len(hour) == 1:
        hour = '0' + hour
    if len(minute) == 1:
        minute = '0' + minute
    return f"{year}-{month}-{day}T{hour}:{minute}"

def scrape_one_old(html, url):
    soup = BeautifulSoup(html, "html.parser")

    # title, date, genre, keyword
    title = soup.find('span', class_="contentTitle").text.strip()
    hmd_ = url.split('/')[-2]
    time_ = soup.find('span', id="news_time").text
    date = make_datetime_normal_old(hmd_, time_)
    genre = [k for k in soup.find('meta', attrs={'name':'keywords'}).get('content').split(',') if k not in ['NHK','ニュース', 'NHK NEWS WEB','ＮＨＫ','ＮＨＫニュース','']]
    
    # article: news_textbody > news_textmore > news_add (paragraph titles are h3)
    article = soup.find(['div','p'], id="news_textbody").text
    if soup.find_all(['div','p'], id="news_textmore") != []:
        for textmore in soup.find_all(['div','p'], id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all(['div','p'], class_="news_add") != []:
        for newsadd in soup.find_all(['div','p'], class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url.split('/')[-1].split('.html')[0],
        'title':title.strip(),
        'article':article.strip(),
        'genre':genre,
        'keywords':[],
        'url':url,
        'datePublished':date,
        'dateModified':""
    }

def get_archiveurl(url):
    driver.get(url)
    time.sleep(5)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(times between|1 time|times).*?<a href="(.+?)">', html)
    #if snap == None:
        #return None
    archiveurl = 'https://web.archive.org' + snap.group(2)
    return archiveurl

def js(dic, year):
    with open(f'nhkweb{year}.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open(f'nhkweb{year}.json', 'w', encoding='utf-8') as f:
        if dic['id'] not in [x['id'] for x in data]:
            data.append(dic)
        else:
            for i, d in enumerate(data):
                if dic['id'] == d['id']:
                    data[i] = dic
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)

def geturl(year=2019):
    idnormal = pd.read_json(f'nhkweb{year}.json', encoding='utf-8')['id'].tolist()
    existurl = pd.read_csv('linknormal.txt', encoding='utf-8', header=None)[0].tolist()
    nolink = pd.read_csv('nolinknormal.txt', encoding='utf-8', header=None)[0].tolist()
    urls = set(existurl) - set(nolink)
    return sorted([url for url in urls if (url.split('.html')[0].split('/')[-1] not in idnormal) and f'html/{year}' in url])

def checkwrongid(): # check wrong ID in newswebeasy
    df = pd.read_json('nhkwebeasy.json', encoding='utf-8')
    print(len(df))
    df['normalID'] = df['url_normal'].apply(lambda x:x.split('/')[-1].strip('.html'))
    return df[df['id'] != df['normalID']]['id'].tolist()

def wrongscrape():
    wrongids = wrongid()
    existurl = pd.read_csv('linknormal.txt', encoding='utf-8', header=None)[0].tolist()[::-1]
    for ID in wrongids:
        for url in existurl:
            if ID in url:
                print(url.split('/*/')[-1])
                break

# scrape

In [34]:
df = pd.read_json('nhkweb2015.json')
df

,id,title,article,genre,keywords,url,datePublished,dateModified
0,k10010239321000,栃木 鹿沼市 ５９世帯に避難指示,栃木県鹿沼市は今後の雨で、土砂災害が発生するおそれがあるとして、すでに避難勧告を出している日...,[社会],[],http://www.nhk.or.jp/news/html/20150917/k10010...,2015-09-17T23:52,
1,k10010000621000,ウィリアム王子 震災の犠牲者に花束,イギリス王室のウィリアム王子は東日本大震災の被災地の宮城県石巻市と女川町を訪れ、震災の犠牲者...,[社会],[],http://www3.nhk.or.jp/news/html/20150301/k1001...,2015-03-01T17:37,
2,k10010000861000,健やかな成長願う「流しびな」,桃の節句を前に、人形作りが盛んなさいたま市岩槻区で子どもの健やかな成長を願って紙のひな人形を...,[科学・文化],[],http://www3.nhk.or.jp/news/html/20150301/k1001...,2015-03-01T23:32,
3,k10010000991000,企業の約４０％ 「正社員不足感じる」,雇用情勢の改善傾向が続くなか、企業の４０％近くが正社員の不足を感じているという調査結果がまと...,[ビジネス],[],http://www3.nhk.or.jp/news/html/20150302/k1001...,2015-03-02T04:16,
4,k10010001851000,ラグビーＷ杯 会場の１２都市が決定,２０１９年に日本で開かれるラグビーワールドカップで試合会場となる１２の都市が発表され、東日本...,[スポーツ],[],http://www3.nhk.or.jp/news/html/20150302/k1001...,2015-03-02T21:52,
...,...,...,...,...,...,...,...,...
782,k10015475931000,チンアナゴなど産卵の撮影成功 世界初,海の底から顔をのぞかせる姿が人気のウナギの仲間、チンアナゴとニシキアナゴ。この産卵の様子の撮...,[科学・文化],[],http://www3.nhk.or.jp/news/html/20150215/k1001...,2015-02-15T19:45,
783,k10015477531000,ネットで税務申告ｅ‐Ｔａｘ手続き簡素化へ,政府は、インターネットを使って税金の申告などができる「ｅーＴａｘ」を使いやすくするため、再来...,[ビジネス],[],http://www3.nhk.or.jp/news/html/20150216/k1001...,2015-02-16T04:12,
784,k10015513531000,リベリア エボラ患者減少で学校再開,エボラ出血熱の流行が続いてきた西アフリカのリベリアで１６日、地元の学校が７か月ぶりに再開し、...,[国際],[],http://www3.nhk.or.jp/news/html/20150217/k1001...,2015-02-17T10:32,
785,k10015770111000,サクラ開花予想 全国的におおむね平年並み,気象会社などが行っている最新のサクラの開花予想によりますと、この春の開花は全国的におおむね平...,[社会],[],http://www3.nhk.or.jp/news/html/20150226/k1001...,2015-02-26T17:04,


In [7]:
df[df['datePublished'].str.startswith('2016-05')]

,id,title,article,genre,keywords,url,datePublished,dateModified
3432,k10010505261000,北アルプス 蓮華岳 男性が滑落し救助要請,長野県と富山県にまたがる北アルプスの蓮華岳で、スキー場ではない斜面を滑るバックカントリースキ...,[社会],[],http://www3.nhk.or.jp/news/html/20160501/k1001...,2016-05-01T06:44,
3433,k10010505271000,北アルプスの奥穂高岳 滑落事故相次ぎ１人意識不明,先月３０日夜、岐阜県の北アルプスの奥穂高岳の岩場近くで、登山者の１人が滑落したという通報があ...,[社会],[],http://www3.nhk.or.jp/news/html/20160501/k1001...,2016-05-01T07:05,
3434,k10010505281000,Ｇ７エネルギー相会合 きょう北九州で開幕,Ｇ７＝主要７か国のエネルギー相会合が１日、北九州市で開幕します。エネルギーの安定調達に向けて...,"[国際, ビジネス]",[サミット],http://www3.nhk.or.jp/news/html/20160501/k1001...,2016-05-01T05:51,
3435,k10010505291000,水俣病の公式確認から６０年 全面的な解決は見通せず,公害の原点と言われる水俣病が公式に確認されてから、１日でちょうど６０年となります。今も新たに...,[社会],[],http://www3.nhk.or.jp/news/html/20160501/k1001...,2016-05-01T08:35,
3436,k10010505301000,首相 きょうから欧州５か国などを歴訪,安倍総理大臣は１日からヨーロッパ５か国などを歴訪し、各国の首脳らと会談することにしています。...,"[国際, 政治]",[サミット],http://www3.nhk.or.jp/news/html/20160501/k1001...,2016-05-01T04:33,
...,...,...,...,...,...,...,...,...
4583,k10010541101000,「ペッタンコ祭」赤ちゃんの成長願いおでこにはんこ,赤ちゃんの健やかな成長を願っておでこにはんこを押す「初山大祭」が群馬県館林市の神社で行われて...,"[地域, 暮らし]",[],http://www3.nhk.or.jp/news/html/20160531/k1001...,2016-05-31T12:14,
4584,k10010541441000,バター不足解消へ 新たに６０００トンの輸入決定,ここ数年、店頭での品薄が続いているバターについて、農林水産省はこの夏は気温が高く原料の生乳の...,"[ビジネス, 暮らし]",[],http://www3.nhk.or.jp/news/html/20160531/k1001...,2016-05-31T16:55,
4585,k10010541451000,クルーズ船から失踪の中国人相次ぐ 不法入国の新手口か,中国の富裕層などが観光に使うクルーズ船の日本への寄港が急増するなか、入国したあと船に戻らずに...,[社会],[],http://www3.nhk.or.jp/news/html/20160531/k1001...,2016-05-31T17:06,
4586,k10010541521000,受動喫煙で死亡 年間１万５０００人と推計,他人のたばこの煙を吸い込む「受動喫煙」によって肺がんや脳卒中などで死亡する人は、国内で年間お...,[社会],[],http://www3.nhk.or.jp/news/html/20160531/k1001...,2016-05-31T17:56,


In [18]:
options = Options()
#options.headless = True
driver = webdriver.Chrome(options=options)

In [36]:
year = 2015
urls = pd.read_csv(f'linknormal.txt', header=None)
urls = sorted(urls[urls[0].str.contains(f'html/{year}')][0].tolist())
id_exist = set(pd.read_json(f'nhkweb{year}.json')['id'].tolist())

for url in urls:
    # check URL
    ID = url.split('.html')[0].split('/')[-1]
    if ID in id_exist:
        continue

    # get archive URL
    archiveurl = get_archiveurl(url)

    # request
    response = requests.get(archiveurl)
    if response.status_code == 200:
        html = response.text
    elif response.status_code == 504:
        response = requests.get(archiveurl)
        if response.status_code == 504:
            continue
        html = response.text
    time.sleep(4)

    # scrape
    url_true = 'htt' + url.split('/htt')[-1]

    try:
        dic = scrape_one_new(html, url_true)
    except:
        dic = scrape_one_old(html, url_true)
    js(dic, year)
    id_exist.add(ID)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=80.0.3987.132)


In [9]:
url_true = 'htt' + driver.current_url.split('/htt')[-1]
html = driver.page_source.encode('utf-8')
try:
    dic = scrape_one_new(html, url_true)
except:
    dic = scrape_one_old(html, url_true)
dic

{'id': 'k10010627901000',
 'title': '台風５号 北日本～東北の太平洋側 暴風・高波に警戒',
 'article': '台風５号は日本の東の海上を北上していて、気象庁は北日本から東北にかけての太平洋側を中心に暴風や高波に警戒するよう呼びかけています。\n気象庁の観測によりますと、台風５号は９日午前６時には、仙台市の東２７０キロの海上を１時間に２５キロの速さで北へ進んでいます。中心の気圧は９８０ヘクトパスカル、最大風速は３０メートル、最大瞬間風速は４０メートルで中心の東側２８０キロ以内と西側１７０キロ以内では風速２５メートル以上の暴風が吹いています。台風はこのあとも北上を続け、９日夜には北海道の東の海上に進み、１０日の朝には千島の近海で温帯低気圧に変わる見込みです。東北の太平洋側では、海はうねりを伴って大しけとなっていて、１０日は北海道の太平洋側でも大しけとなる見込みです。また、北海道の太平洋側では９日夜にかけて非常に強い風が吹き、最大風速は２０メートル、最大瞬間風速は３０メートルに達すると予想されています。気象庁は北日本から東北にかけての太平洋側を中心に、暴風や高波に警戒するよう呼びかけています。',
 'genre': ['気象', '災害'],
 'keywords': [],
 'url': 'http://www3.nhk.or.jp/news/html/20160809/k10010627901000.html',
 'datePublished': '2016-08-09T06:29',
 'dateModified': ''}

In [10]:
js(dic, year)

# clean

In [33]:
# check category

year = 2017

with open(f'nhkweb{year}.json','r', encoding='utf-8') as f:
    data = json.load(f)
print('articles: ', len(data))
genre = Counter()
for dic in data:
    for g in dic['genre']:
        genre[g] += 1
genre.most_common()

articles:  10740


[('国際', 4223),
 ('社会', 2827),
 ('政治', 1525),
 ('ビジネス', 1341),
 ('スポーツ', 943),
 ('気象・災害', 792),
 ('科学・文化', 783),
 ('暮らし', 519),
 ('地域', 423)]

In [31]:
# genre <> keywords
with open(f'nhkweb{year}.json','r', encoding='utf-8') as f:
    data = json.load(f)

category = ['社会', '国際', 'ビジネス', 'スポーツ', '政治', '科学・文化', '暮らし', '地域', '気象・災害']
for i, dic in enumerate(data):
    newgenre = []
    newkey = []
    for j in dic['genre']:
        if j in category:
            newgenre.append(j)
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        elif j == "科学・医療" or j == "文化・エンタメ" or j == "科学":
            newgenre.append('科学・文化')
        elif j == "暮らし文化":
            newgenre.append('暮らし')
            newgenre.append('科学・文化')
        elif j == "経済":
            newgenre.append('ビジネス')
        else:
            newkey.append(j)
    for j in dic['keywords']:
        if j in category:
            newgenre.append(j)
        elif j == "科学・医療" or j == "文化・エンタメ" or j == "科学":
            newgenre.append('科学・文化')
        elif j == "暮らし文化":
            newgenre.append('暮らし')
            newgenre.append('科学・文化')
        elif j == "災害" or j == "気象":
            newgenre.append('気象・災害')
        else:
            newkey.append(j)
    data[i]['genre'] = list(set(newgenre))
    data[i]['keywords'] = list(set(newkey))

with open(f'nhkweb{year}.json','w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
    

In [32]:
pd.read_json(f'nhkweb{year}.json').id.value_counts()

k10011141491000    1
k10011000931000    1
k10010934981000    1
k10010951261000    1
k10010933321000    1
                  ..
k10011177701000    1
k10011195271000    1
k10010962891000    1
k10011073211000    1
k10011211931000    1
Name: id, Length: 10740, dtype: int64